In [ ]:
from dna_features_viewer import GraphicFeature, GraphicRecord
from foreground import get_foreground 
import matplotlib.pyplot as plt
import matplotlib.patches as patches
figure_width = 15
total_length = 42000
# nb_plots = len(to_plot)
nb_plots = 5
fig, subplots  = plt.subplots(
    nb_plots+1, 1, figsize=(figure_width, 3+.2*nb_plots), sharex=True, gridspec_kw={'height_ratios': [20] +[1]*nb_plots })
#plt.subplots_adjust(hspace  = 5)
subplot_i = 1

def hide_axes(ax,genome_start_pos,genome_end_pos):
    #hack: since I don't~~~~~ didn't know how to erase axes around coordinates, draw a white box over them.
    # is that a ugly hack? yes. does it render well? also yes
    ax.add_patch(
     patches.Rectangle(
        (-100, -0.25),
        genome_start_pos,
        0.5,
        fill=True,      # remove background
        facecolor='white'
     ) ) 
    ax.add_patch(
     patches.Rectangle(
        (genome_end_pos, -0.25),
        total_length+100,
        0.5,
        fill=True,      # remove background
        facecolor='white'
     ) ) 
    
# plot the virtual tracks for domain annotations
for virtual_track, v_subplot_i, annotation in \
    [(sc2_virtual_track,  0,          "SARS-CoV-2"), 
     #(nido_virtual_track, nb_plots+1, "Nido HMM")
    ]:
    record = GraphicRecord(sequence_length=total_length, feature_level_height=1, features=virtual_track)
    ax, _ = record.plot(ax=subplots[v_subplot_i], figure_width=figure_width)
    ax.set_facecolor("white")
    ax.annotate(annotation, (-2300,-0.1), xycoords='data', annotation_clip=False) #https://stackoverflow.com/questions/18537879/matplotlib-how-to-write-annotation-outside-the-drawing-in-data-coords
    ax.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False) # remove ticks
    genome_start_pos = rdrp_fixed_position - [x[0] for x in sars_cov_2_features if "RdRP" in x[3]][0]
    genome_end_pos = genome_start_pos + 29903
    hide_axes(ax,genome_start_pos,genome_end_pos)


all_names = set() # this is to keep track of all domains we encounter
#for accession, why in sorted(list(to_plot),key = lambda x:x[1])[:nb_plots]:
for accession in to_plot:
    
    #determine position of rdrp
    list_rdrp_positions = [x[0] for x in dgs[accession] if (x[3] == 'RdRP_1' \
                                                            and (accession not in multicontig_assemblies \
                                                                 or multicontig_assemblies[accession] in x[4]\
                                                                )\
                                                           )]

    if len(list_rdrp_positions) > 1:
        print("accession",accession,"has multiple rdrp's but no idea how to disambiguate")
        break
    elif len(list_rdrp_positions) == 1:
        rdrp_genome_position = list_rdrp_positions[0]
        rdrp_offset = rdrp_fixed_position - rdrp_genome_position
    else:
        print("accession",accession,"rdrp not found: ",[(x[3],x[4]) for x in dgs[accession]])
        rdrp_offset = 0
        rdrp_genome_position=rdrp_fixed_position
    
    features = []
    ctgs = set()
    for feature in dgs[accession]:
        start, end, strand, name, ctg, is_complete, pfam_hmm = feature
        
        if accession in multicontig_assemblies:
            if multicontig_assemblies[accession] not in ctg: 
                # select only 1 contig in multi-contig assemblies
                continue
        ctgs.add(ctg[:-2])

        all_names.add(name)
        start, end = start+rdrp_offset, end+rdrp_offset # apply offset to line up rdrp's
        if name in dpalette:
            color = dpalette[name]
        elif pfam_hmm == "nido":
            color = nidopalette[nido_exclusives.index(name)]
        else:
            color = "red"
        linewidth = 0 if pfam_hmm == "sc2" else 1
        linecolor = "#EE0000"
        #print(accession,name,start,end)
        features += [GraphicFeature(start=start, end=end, strand=int(strand+"1"), color = color, 
                                    linewidth=linewidth, linecolor=linecolor, thickness=7)]
    if len(ctgs) > 1:
        print("warning!",accession,"has",len(ctgs),"contigs")
        
    record = GraphicRecord(sequence_length=total_length, feature_level_height=0, features=features)
    ax, _ = record.plot(ax=subplots[subplot_i], figure_width=figure_width)
    ax.set_facecolor("white")
    ax.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False) # remove ticks
    
    # display left text
    #https://stackoverflow.com/questions/18537879/matplotlib-how-to-write-annotation-outside-the-drawing-in-data-coords
    ax.annotate(("%15s" % taxlabels[accession]) ,(-4000,-2), xycoords='data', annotation_clip=False, ha='left') 
    #ax.annotate(('%6d nt' % genome_lengths[accession]) ,(-3500,-0.1), xycoords='data', annotation_clip=False, ha='right') 
    #ax.annotate(('OTU=%-12s' % why) ,(-2300,-0.1), xycoords='data', annotation_clip=False, ha='left') 
    if accession in descs:
        ax.annotate(('%s' % descs[accession]) ,(-5000,-0.1), xycoords='data', annotation_clip=False, ha='right') 
    
    # hide axis before/after true assembly positions
    genome_start_pos = rdrp_fixed_position - rdrp_genome_position
    genome_end_pos = genome_start_pos + genome_lengths[accession]
    hide_axes(ax,genome_start_pos,genome_end_pos)
    
    if accession == "SRR9643845":
        # surgery to indicate scaffolding
        ax.add_patch(
         patches.Rectangle(
            (rdrp_fixed_position + (12840 - rdrp_genome_position), -2.1),
            16272-12840,
            4,
            fill=True,      # remove background
            facecolor='white'
         ) ) 
    
    subplot_i += 1
    
    
fig.tight_layout(pad=-1)
fig.savefig('genome_structure.pdf', bbox_inches='tight')
fig.savefig('genome_structure.svg', bbox_inches='tight')